In [3]:
# | default_exp agent.planner.task_executor

In [1]:
# | export
import json
from typing import Optional, List, Tuple
from datetime import datetime
from pathlib import Path
from rich.console import Console

from agentic.agent.debater import create_debate
from agentic.tools.introspect import IntrospectTool
from agentic.tools.base import ToolMetadata, ToolCategory

from agentic.agent.planner.models import (
    Task, ActionStep, ActionResult, IntrospectionResult, 
    TaskExecutionResult, ExecutionStatus
)

In [2]:
# | export

class TaskExecutor:
    """Handles task execution with introspection and retries"""
    
    def __init__(self, agent, console: Console, max_retries: int = 3):
        self.agent = agent
        self.console = console
        self.max_retries = max_retries
        self.introspect = IntrospectTool(
            metadata=ToolMetadata(
                name="introspect",
                description="Task validation and feedback tool",
                category=ToolCategory.INTELLIGENCE
            )
        )
    
    async def execute_task(self, task: Task) -> TaskExecutionResult:
        """Execute all actions in a task with full tracking"""
        start_time = datetime.now()
        self.console.print(f"\n🔄 Executing Task: {task.name}")
        
        if task.needs_debate:
            self.console.print("🗣️ Initiating debate for decision making...")
            debate_context = f"Task: {task.description}\nOptions to consider: {', '.join(task.potential_options)}"
            debate_result = await create_debate(topic=task.name, context=debate_context)
            self.console.print(f"Debate result: {debate_result}")
        
        task_result = TaskExecutionResult(
            task_id=task.id,
            task_name=task.name,
            status=ExecutionStatus.IN_PROGRESS,
            execution_time=0.0,
            success_criteria_met=False,
            artifacts_created=[]
        )
        
        overall_success = True
        
        for action in task.actions:
            self.console.print(f"\n  📌 Step {action.step}: {action.purpose}")
            
            action_result, introspection_result = self._execute_action_with_retries(task, action)
            
            task_result.actions_executed.append(action_result)
            if introspection_result:
                task_result.introspection_results.append(introspection_result)
            
            # Update artifacts (ensure it's a list)
            if not isinstance(task_result.artifacts_created, list):
                task_result.artifacts_created = []
                
            artifacts = action_result.artifacts_created
            if isinstance(artifacts, list):
                task_result.artifacts_created.extend(artifacts)
            elif artifacts:
                task_result.artifacts_created.extend([str(artifacts)])
            
            if action_result.status != ExecutionStatus.SUCCESS:
                self.console.print(f"  ❌ Step {action.step} failed after {self.max_retries} retries")
                overall_success = False
                break
            else:
                self.console.print(f"  ✅ Step {action.step} completed successfully")
        
        # Finalize task result
        end_time = datetime.now()
        task_result.execution_time = (end_time - start_time).total_seconds()
        task_result.status = ExecutionStatus.SUCCESS if overall_success else ExecutionStatus.FAILED
        task_result.success_criteria_met = overall_success
        
        return task_result
    
    def _execute_action_with_retries(self, task: Task, action: ActionStep) -> Tuple[ActionResult, Optional[IntrospectionResult]]:
        """Execute single action with introspection and retries"""
        
        action_result = ActionResult(
            action_id=f"{task.id}_step_{action.step}",
            tool_used="agent",
            result="",
            execution_time=0.0,
            status=ExecutionStatus.PENDING,
            artifacts_created=[]
        )
        
        introspection_result = None
        retry_feedback = ""
        
        for attempt in range(self.max_retries):
            self.console.print(f"    🔄 Attempt {attempt + 1}/{self.max_retries}")
            
            try:
                start_time = datetime.now()
                
                self.console.print(f"    🚀 Executing action: {action.purpose}")
                
                result = self._execute_single_action(action, retry_feedback, attempt + 1)
                
                end_time = datetime.now()
                execution_time = (end_time - start_time).total_seconds()
                
                self.console.print(f"    📤 Action result length: {len(str(result))} chars")
                self.console.print(f"    📤 Action result preview: {str(result)[:200]}...")
                
                action_result.result = result
                action_result.execution_time = execution_time
                action_result.status = ExecutionStatus.SUCCESS
                
                new_artifacts = self._detect_new_artifacts()
                action_result.artifacts_created = new_artifacts if isinstance(new_artifacts, list) else []
                
                self.console.print(f"    📝 ActionStep completed in {execution_time:.2f}s")
                if action_result.artifacts_created:
                    if len(action_result.artifacts_created) <= 3:
                        self.console.print(f"    📁 New artifacts: {action_result.artifacts_created}")
                    else:
                        shown = action_result.artifacts_created[:3]
                        remaining = len(action_result.artifacts_created) - 3
                        self.console.print(f"    📁 New artifacts: {shown} ... (+{remaining} more)")
                else:
                    self.console.print(f"    📁 New artifacts: []")
                
                if action.introspect_after:
                    self.console.print(f"    🔍 Starting introspection...")
                    introspection_result = self._introspect_action(task, action, result)
                    
                    if introspection_result.success:
                        self.console.print(f"    ✅ Introspection passed (score: {introspection_result.score})")
                        return action_result, introspection_result
                    else:
                        self.console.print(f"    ⚠️ Introspection failed (score: {introspection_result.score})")
                        self.console.print(f"    💬 Failure reason: {introspection_result.feedback}")
                        if attempt < self.max_retries - 1:
                            action_result.status = ExecutionStatus.RETRYING
                            retry_feedback = introspection_result.feedback
                            self.console.print(f"    🔄 Retrying with feedback: {retry_feedback}")
                            continue
                else:
                    self.console.print(f"    ⏭️ Skipping introspection (not required)")
                    return action_result, None
                    
            except Exception as e:
                self.console.print(f"    ❌ ActionStep execution error: {e}")
                import traceback
                self.console.print(f"    📋 Full traceback: {traceback.format_exc()}")
                action_result.error_message = str(e)
                action_result.status = ExecutionStatus.FAILED
                
                if attempt < self.max_retries - 1:
                    action_result.status = ExecutionStatus.RETRYING
                    retry_feedback = f"Previous attempt failed with error: {str(e)}"
                    continue
        
        action_result.status = ExecutionStatus.FAILED
        return action_result, introspection_result
    
    def _execute_single_action(self, action: ActionStep, retry_feedback: str = "", attempt: int = 1) -> str:
        """Execute single action using Agent's tool system"""
        
        retry_guidance = ""
        if attempt > 1:
            if attempt == 2:
                retry_guidance = "RETRY GUIDANCE: Previous attempt failed. Try the same approach with corrections."
            elif attempt >= 3:
                retry_guidance = "RETRY GUIDANCE: Multiple failures detected. SWITCH TO ALTERNATIVE APPROACH. Don't repeat the same failing method."
        
        enriched_prompt = f"""
SYSTEM CONTEXT:
{action.system_prompt}

USER REQUEST:
{action.user_prompt}

EXECUTION DETAILS:
- Purpose: {action.purpose}
- Sub-steps: {', '.join(action.sub_steps)}
- Execution Mode: {action.execution_mode}

{retry_guidance}

{f"RETRY FEEDBACK: {retry_feedback}" if retry_feedback else ""}

Execute this action and provide detailed output. If a tool is specified, use it to perform the actual operation.
"""
        
        result = self.agent.run(enriched_prompt, stream=True, max_iterations=5)
        return result.get("content", "")
    
    def _introspect_action(self, task: Task, action: ActionStep, result: str) -> IntrospectionResult:
        """Use introspect tool to validate action success"""
        
        self.console.print(f"    🔍 Starting introspection for step {action.step}")
        
        try:
            introspect_params = {
                "action": "task_review",
                "task_review": {
                    "task_id": task.id,
                    "task_description": f"{task.description} - Step {action.step}: {action.purpose}",
                    "solution_provided": result,
                    "success_criteria": task.success_criteria,
                    "actual_result": result,
                    "execution_time": 1.0,
                    "retry_count": 0
                }
            }
            
            self.console.print(f"    📋 Introspect params: {introspect_params}")
            
            introspect_result = self.introspect.execute(**introspect_params)
            
            self.console.print(f"    📊 Raw introspect result: {introspect_result}")
            
            if introspect_result and 'performance_score' in introspect_result:
                score = introspect_result['performance_score']
                success = introspect_result.get('success', False)
                feedback = introspect_result.get('feedback_for_retry', 'No feedback provided')
                next_action = introspect_result.get('next_action', 'proceed' if success else 'retry')
                recommendations = introspect_result.get('recommendations', [])
                
                self.console.print(f"    ✅ Parsed introspection - Score: {score}, Success: {success}")
                self.console.print(f"    💬 Feedback: {feedback}")
                
                return IntrospectionResult(
                    success=success,
                    score=score,
                    feedback=feedback,
                    next_action=next_action,
                    recommendations=recommendations
                )
            else:
                self.console.print(f"    ❌ Invalid introspect result format: {introspect_result}")
                
        except Exception as e:
            self.console.print(f"    ⚠️ Introspection error: {e}")
            import traceback
            self.console.print(f"    📋 Full traceback: {traceback.format_exc()}")
        
        return IntrospectionResult(
            success=False,
            score=0.0,
            feedback="Introspection failed - see logs for details",
            next_action="retry",
            recommendations=["Fix introspection system"]
        )
    
    def _detect_new_artifacts(self) -> List[str]:
        """Detect newly created files/artifacts"""
        current_files = set()
        current_dir = Path(".")
        
        exclude_patterns = {
            '.venv', '__pycache__', '.git', 'node_modules', 
            '.pytest_cache', '.mypy_cache', 'execution_cache'
        }
        
        for file_path in current_dir.glob("**/*"):
            if file_path.is_file() and not file_path.name.startswith('.'):
                if any(part in exclude_patterns for part in file_path.parts):
                    continue
                if file_path.suffix in {'.pyc', '.pyo', '.log', '.tmp'}:
                    continue
                current_files.add(str(file_path))
        
        return list(current_files)
